In [2]:
import os, sys

# find repo root (looks for liars_poker/ or pyproject.toml)
def find_repo_root(start_dir: str) -> str:
    cur = os.path.abspath(start_dir)
    for _ in range(6):
        if os.path.isdir(os.path.join(cur, "liars_poker")) or os.path.exists(os.path.join(cur, "pyproject.toml")):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            break
        cur = parent
    return os.path.abspath(os.path.join(start_dir, "..", ".."))

NB_DIR = os.getcwd()
REPO_ROOT = find_repo_root(NB_DIR)
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

ARTIFACTS_ROOT = os.path.join(REPO_ROOT, "artifacts")
os.makedirs(ARTIFACTS_ROOT, exist_ok=True)

print("repo root   :", REPO_ROOT)
print("artifacts   :", ARTIFACTS_ROOT)


repo root   : c:\Users\adidh\Documents\liars_poker
artifacts   : c:\Users\adidh\Documents\liars_poker\artifacts


In [3]:
import scipy.stats as stats
import random
from pprint import pprint
import numpy as np
from liars_poker import (
    GameSpec, Env, InfoSet, Rules,
    Policy, TabularPolicy, CommitOnceMixture, RandomPolicy,
    eval_both_seats
)

from liars_poker.algo.br_exact import best_response_exact
from liars_poker.eval.match import exact_eval_tabular_both_seats
from typing import List, Tuple

SEED = 42
random.seed(SEED)

# small game; P1 always starts by design
spec = GameSpec(ranks=4, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=True)
rules = Rules(spec)


In [4]:
def _clean_simplex(w, tol=1e-18):
    """
    Project-ish cleanup: clip tiny negatives to 0, drop tiny entries, renormalize.
    If sum becomes 0 (all tiny), fallback to uniform over all entries.
    """
    w = np.asarray(w, dtype=float)
    w = np.where(w < tol, 0.0, w)          # clip tiny negatives to 0
    s = w.sum()
    if s <= tol:
        # fallback: uniform distribution
        w = np.full_like(w, 1.0 / len(w))
    else:
        w = w / s
    return w

def flatten_commit_once(policy: Policy) -> List[Tuple[Policy, float]]:
    if isinstance(policy, CommitOnceMixture):
        return list(zip(policy.policies, policy.weights))
    return [(policy, 1.0)]

def mix_policies(base_policy: Policy, br_policy: Policy, eta: float, rng: random.Random | None = None) -> CommitOnceMixture:
    base_components = flatten_commit_once(base_policy)
    br_components = flatten_commit_once(br_policy)

    combined_policies: List[Policy] = []
    combined_weights: List[float] = []

    for policy, weight in base_components:
        scaled = (1.0 - eta) * weight
        combined_policies.append(policy)
        combined_weights.append(scaled)

    for policy, weight in br_components:
        scaled = eta * weight
        combined_policies.append(policy)
        combined_weights.append(scaled)

    mixed_policy = CommitOnceMixture(combined_policies, combined_weights, rng=rng)
    mixed_policy.bind_rules(base_policy._rules)

    return mixed_policy

def mix_multiple_policies(policies: List[object], weights: List[float], rng: "random.Random | None" = None):
    """
    Mix an arbitrary list of policies with weights (sum need not be 1; will be normalized).
    Returns a CommitOnceMixture with base rules taken from the first policy.
    """
    assert len(policies) == len(weights) and len(policies) > 0, "policies/weights must be same nonzero length"
    total = float(sum(weights))
    if total <= 0:
        raise ValueError("weights must sum to > 0")
    norm_w = [w / total for w in weights]

    # Flatten any CommitOnceMixture items to keep a "flat" mixture
    flat_pols: List[object] = []
    flat_wts: List[float] = []
    for pol, w in zip(policies, norm_w):
        comps = flatten_commit_once(pol)
        for sub_pol, sub_w in comps:
            flat_pols.append(sub_pol)
            flat_wts.append(w * sub_w)

    # Normalize again after flattening (in case nested mixtures existed)
    tot2 = float(sum(flat_wts))
    flat_wts = [w / tot2 for w in flat_wts]

    mixed = CommitOnceMixture(flat_pols, flat_wts, rng=rng)
    mixed.bind_rules(policies[0]._rules)  # assume same rules across population
    return mixed

In [5]:
a0 = RandomPolicy()
a0.bind_rules(rules=rules)

In [ ]:
# all_averages = [a0]
# all_brs = []

# curr_av = a0

In [ ]:
# b_i, br_computer = best_response_exact(spec=spec, policy=curr_av)
# x, y = br_computer.exploitability()
# print((x+y)/2)




In [ ]:


# exact_eval_tabular_both_seats(spec, b_i, curr_av)

In [ ]:
# curr_av = b_i

In [6]:
import numpy as np
from typing import List, Tuple, Dict, Any
from scipy.optimize import linprog



# --- Exact payoff between two tabular (or mixture) policies ---
def payoff(spec, row_policy, col_policy) -> float:
    """
    Returns the zero-sum payoff to the row player (P1) using your exact evaluator.
    If policies are mixtures, they can be used directly or converted to tabular.
    """
    # Ensure tabular for evaluation (your evaluator is tabular-friendly)
    rp = row_policy.to_tabular() if hasattr(row_policy, "to_tabular") else row_policy
    cp = col_policy.to_tabular() if hasattr(col_policy, "to_tabular") else col_policy
    #res: Dict[str, Any] = exact_eval_tabular_both_seats(spec, p1=rp, p2=cp)
    res: Dict[str, Any] = eval_both_seats(spec, p1=rp, p2=cp, episodes=100)
    # Assuming res like {'P1': 0.96, 'P2': 0.04} where P1 is the row player's success prob/value
    return float(res['P1'])

# --- Maintain the restricted meta-game payoff matrix incrementally ---
def extend_payoff_matrix(spec, P: np.ndarray, S1: List[object], S2: List[object],
                         new_row_idxs=None, new_col_idxs=None) -> np.ndarray:
    if new_row_idxs is None: new_row_idxs = range(len(S1))
    if new_col_idxs is None: new_col_idxs = range(len(S2))
    for i in new_row_idxs:
        for j in new_col_idxs:
            P[i, j] = payoff(spec, S1[i], S2[j])
    return P

# --- Zero-sum matrix solver: find (x*, y*, v) with LPs ---
def solve_minimax_row(P: np.ndarray):
    """
    Row player LP:
      maximize v  s.t. x in simplex, and for all columns j: sum_i x_i * P[i,j] >= v
    Implemented as minimize -v.
    """
    m, n = P.shape
    c = np.concatenate([np.zeros(m), [-1.0]])  # minimize -v
    A_ub, b_ub = [], []
    for j in range(n):
        row = np.concatenate([-P[:, j], [1.0]])  # -P^T x + 1*v <= 0   (i.e., P^T x >= v)
        A_ub.append(row); b_ub.append(0.0)
    A_eq = [np.concatenate([np.ones(m), [0.0]])]
    b_eq = [1.0]
    bounds = [(0.0, None)] * m + [(None, None)]
    res = linprog(c, A_ub=np.array(A_ub), b_ub=np.array(b_ub),
                  A_eq=np.array(A_eq), b_eq=np.array(b_eq),
                  bounds=bounds, method="highs")
    if not res.success:
        raise RuntimeError(f"Row LP failed: {res.message}")
    x = res.x[:m]
    v = res.x[-1]
    return x, v

def solve_minimax_col(P: np.ndarray):
    """
    Column player LP (best defense):
      minimize w  s.t. y in simplex, and for all rows i: sum_j P[i,j] * y_j <= w
    Implemented directly.
    """
    m, n = P.shape
    c = np.concatenate([np.zeros(n), [1.0]])  # minimize w
    A_ub, b_ub = [], []
    for i in range(m):
        row = np.concatenate([P[i, :], [-1.0]])  # P[i,:] y - w <= 0
        A_ub.append(row); b_ub.append(0.0)
    A_eq = [np.concatenate([np.ones(n), [0.0]])]
    b_eq = [1.0]
    bounds = [(0.0, None)] * n + [(None, None)]
    res = linprog(c, A_ub=np.array(A_ub), b_ub=np.array(b_ub),
                  A_eq=np.array(A_eq), b_eq=np.array(b_eq),
                  bounds=bounds, method="highs")
    if not res.success:
        raise RuntimeError(f"Col LP failed: {res.message}")
    y = res.x[:n]
    w = res.x[-1]
    return y, w

def solve_minimax(P: np.ndarray):
    """
    Solve zero-sum matrix game. Returns (x*, y*, v_R).
    v_R should be equal to both row's v and column's w up to solver tolerance.
    """
    x, v_row = solve_minimax_row(P)
    y, w_col = solve_minimax_col(P)
    x = _clean_simplex(x)
    y = _clean_simplex(y)
    # Reconcile tiny numerical discrepancies by averaging
    v_R = 0.5 * (v_row + w_col)
    return x, y, v_R

# --- Bounds for DO (exact BRs) ---
def bounds_exact(spec, S1: List[object], S2: List[object], x: np.ndarray, y: np.ndarray):
    """
    Build mixtures sigma1, sigma2 over S1,S2 using mix_multiple_policies,
    compute exact BRs against them, and return (L, U, b1, b2, sigma1, sigma2).
    """
    sigma1 = mix_multiple_policies(S1, x).to_tabular()
    sigma2 = mix_multiple_policies(S2, y).to_tabular()

    b1, _ = best_response_exact(spec=spec, policy=sigma2)  # row's BR to sigma2
    b2, _ = best_response_exact(spec=spec, policy=sigma1)  # col's BR to sigma1

    # Upper/lower bounds on true value v*:
    U = payoff(spec, b1, sigma2)        # >= v*
    L = payoff(spec, sigma1, b2)        # <= v*
    return L, U, b1, b2, sigma1, sigma2

# --- Double Oracle main ---
def double_oracle(spec, init_policy, eps: float = 1e-3, max_iters: int = 100, verbose: bool = True):
    """
    Minimal Double Oracle for 2p zero-sum.
    - spec: your game spec
    - init_policy: starting policy (will seed both S1 and S2)
    - eps: stop when (U - L) <= eps
    Returns dict with populations, payoff matrix, final mixtures, value, and log.
    """
    # Initialize populations
    S1: List[object] = [init_policy.to_tabular() if hasattr(init_policy, "to_tabular") else init_policy]
    S2: List[object] = [init_policy.to_tabular() if hasattr(init_policy, "to_tabular") else init_policy]

    # Initialize payoff matrix
    P = np.full((1, 1), np.nan, dtype=float)
    P = extend_payoff_matrix(spec, P, S1, S2)

    history = []
    for t in range(max_iters):
        print(f"starting {t}")

        # 1) Solve restricted meta-game
        x, y, vR = solve_minimax(P)
        print("solved meta nash")

        # 2) Compute bounds and exact BRs
        L, U, b1, b2, sigma1, sigma2 = bounds_exact(spec, S1, S2, x, y)
        gap = U - L
        history.append(dict(t=t, vR=float(vR), L=float(L), U=float(U), gap=float(gap)))
        if verbose:
            print(f"[DO {t:03d}] vR={vR:.8f}  L={L:.8f}  U={U:.8f}  gap={gap:.8f}  |S1|={len(S1)} |S2|={len(S2)}")

        # 3) Check termination
        if gap <= eps:
            break

        # 4) Grow populations (no duplicate checks, per your request)
        S1.append(b1.to_tabular() if hasattr(b1, "to_tabular") else b1)
        S2.append(b2.to_tabular() if hasattr(b2, "to_tabular") else b2)

        # 5) Expand payoff matrix with new row/col, and fill only what's new
        P = np.pad(P, ((0, 1), (0, 1)), mode="constant", constant_values=np.nan)
        # New row vs all old+new columns (including the new column we will fill below)
        P = extend_payoff_matrix(spec, P, S1, S2, new_row_idxs=[len(S1) - 1], new_col_idxs=range(len(S2)))
        # All rows vs new column (some already filled by previous line; harmless to recompute)
        P = extend_payoff_matrix(spec, P, S1, S2, new_row_idxs=range(len(S1)), new_col_idxs=[len(S2) - 1])

    # Final restricted equilibrium
    x, y, vR = solve_minimax(P)
    sigma1 = mix_multiple_policies(S1, x).to_tabular()
    sigma2 = mix_multiple_policies(S2, y).to_tabular()

    return dict(S1=S1, S2=S2, P=P, sigma1=sigma1, sigma2=sigma2, value=float(vR), log=history)


In [7]:
res = double_oracle(spec, a0, max_iters=1000, eps=0)

starting 0
solved meta nash
[DO 000] vR=0.47000000  L=0.22000000  U=0.77000000  gap=0.55000000  |S1|=1 |S2|=1
starting 1
solved meta nash
[DO 001] vR=0.46000000  L=0.31000000  U=0.66000000  gap=0.35000000  |S1|=2 |S2|=2
starting 2
solved meta nash
[DO 002] vR=0.47000000  L=0.15000000  U=0.90000000  gap=0.75000000  |S1|=3 |S2|=3
starting 3
solved meta nash
[DO 003] vR=0.48649591  L=0.30000000  U=0.76000000  gap=0.46000000  |S1|=4 |S2|=4
starting 4
solved meta nash
[DO 004] vR=0.55000000  L=0.20000000  U=0.75000000  gap=0.55000000  |S1|=5 |S2|=5
starting 5
solved meta nash
[DO 005] vR=0.48462986  L=0.36000000  U=0.61000000  gap=0.25000000  |S1|=6 |S2|=6
starting 6
solved meta nash
[DO 006] vR=0.52392857  L=0.19000000  U=0.73000000  gap=0.54000000  |S1|=7 |S2|=7
starting 7
solved meta nash
[DO 007] vR=0.51281688  L=0.40000000  U=0.72000000  gap=0.32000000  |S1|=8 |S2|=8
starting 8
solved meta nash
[DO 008] vR=0.49157381  L=0.38000000  U=0.64000000  gap=0.26000000  |S1|=9 |S2|=9
starting 9

In [8]:
b_i, br_computer = best_response_exact(spec=spec, policy=res['sigma2']) 

In [ ]:
p_first, p_second = br_computer.exploitability()
predicted = 0.5 * (p_first + p_second)
print(f'{p_first}, {p_second}, {predicted}')

0.5825178665486228, 0.5535129755017114, 0.5680154210251671


In [ ]:
# res['sigma2'].store_efficiently('/root/liars_poker/artifacts/runs/run_temp_254')